In [9]:
# notebook to remove images that do not occur within 5 minutes of a bird detection

In [1]:
from PIL import Image
import glob
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
%matplotlib inline

In [2]:
bird_files = sorted(glob.glob('detr_results_*[0-9].txt'))
all_imfiles = []

In [3]:
for f in bird_files:
    print(f)
    
    # extract timestamp
    dt = f.rstrip('.txt').split('_')[-1]

    # identify images from this day
    imfiles = sorted(glob.glob(f'images/{dt}_*.jpg'))
    imdata = pd.DataFrame(imfiles, columns=['filename'])
    imdata['tstamp'] = pd.to_datetime(imfiles, format='images/%Y%m%d_%H%M%S%f.jpg')
    imdata['keep'] = 0

    # read bird data
    df = pd.read_csv(f, delimiter=';')
    df['tstamp'] = pd.to_datetime(df['filename'], format='images/%Y%m%d_%H%M%S%f.jpg')

    # iterate over detected birds
    for i, row in df.iterrows():
        # keep images within 5 minutes of detected bird
        ix = np.logical_and(
            imdata['tstamp'] > (row['tstamp'] - pd.Timedelta(minutes=5)),
            imdata['tstamp'] < (row['tstamp'] + pd.Timedelta(minutes=5))
        )
        # print(ix.sum())
        imdata.loc[ix, 'keep'] = 1

    # ensure that every detected bird is marked as keep
    assert np.in1d(df['filename'], imdata.query('keep == 1')['filename']).all()
    all_imfiles.append(imdata)

detr_results_20240212.txt
detr_results_20240213.txt
detr_results_20240214.txt
detr_results_20240215.txt
detr_results_20240216.txt
detr_results_20240217.txt
detr_results_20240218.txt
detr_results_20240219.txt
detr_results_20240220.txt
detr_results_20240221.txt
detr_results_20240222.txt
detr_results_20240223.txt
detr_results_20240224.txt
detr_results_20240225.txt
detr_results_20240226.txt
detr_results_20240227.txt
detr_results_20240228.txt
detr_results_20240229.txt
detr_results_20240301.txt
detr_results_20240302.txt


In [4]:
imdata = pd.concat(all_imfiles).reset_index(drop=True)

In [5]:
imdata['fsize'] = [os.stat(f).st_size for f in imdata['filename']]

In [6]:
imdata.groupby('keep').agg({
    'filename': 'count',
    'fsize': 'sum'
})

,filename,fsize
keep,,
0,341597,238241989072
1,301311,239607747250


In [7]:
for i, row in imdata.query('keep == 0').iterrows():
    os.rename(row['filename'], row['filename'].replace('images', 'trash'))

In [8]:
# 182156